In [1]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import SVD

In [2]:
# Load sample data from CSV files
meals_df = pd.read_csv('meals.csv')
users_df = pd.read_csv('users.csv')
ratings_df = pd.read_csv('ratings.csv')

In [3]:
# meals_df.head()

In [4]:
# users_df.head()

In [5]:
# ratings_df.head()

# Collaborative based filtering
This approach just uses ratings data

In [6]:
# Surprise Reader object specifies rating scale
reader = Reader(rating_scale=(1, 5))

# Load the data into the Surprise Dataset format
data = Dataset.load_from_df(ratings_df[['user_id', 'meal_id', 'rating']], reader)

# Split data into train, test sets
trainset, testset = train_test_split(data, test_size=0.25)

# Train user-user collaborative filtering model - in this case KNNBasic)
user_user_algo = KNNBasic(sim_options={'user_based': True})
user_user_algo.fit(trainset)

# Train item-item collaborative filtering model - in this case KNNBasic
item_item_algo = KNNBasic(sim_options={'user_based': False})
item_item_algo.fit(trainset)


# Train a user-item collaborative filtering model - in this case SVD
user_item_algo = SVD()
user_item_algo.fit(trainset) 

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


Hybrid approach will use recommendations from both user-user and item-item collaborative filters, and weight them to come up with final recommendation.

In [7]:
num_meals = len(meals_df)
num_users = len(users_df)
num_ratings = len(ratings_df)

def user_item_collaborative_filtering(user_id, num_recommendations=5):
    # Get list of all meal IDs
    all_meal_ids = [meal_id for meal_id in range(1, num_meals + 1)]
    print("all_meal_ids", all_meal_ids)
    # Remove meal IDs that the user has already rated
    user_rated_meals = ratings_df[ratings_df['user_id'] == user_id]['meal_id'].tolist()
    remaining_meal_ids = list(set(all_meal_ids) - set(user_rated_meals))
    print("user_rated_meals", user_rated_meals)
    print("remaining_meal_ids", remaining_meal_ids)
    # Predict ratings for remaining meals
    predictions = [(meal_id, user_item_algo.predict(user_id, meal_id).est) for meal_id in remaining_meal_ids]
    print("predictions", predictions)
    # Sort predictions by estimated rating in descending order
    sorted_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)
    print("sorted_predictions", sorted_predictions)
    # Get the top N recommendations
    top_recommendations = [meal_id for meal_id, _ in sorted_predictions[:num_recommendations]]
    print("top_recommendations", top_recommendations)
    return top_recommendations


In [8]:

# User ID and number of recommendations
user_id = 48
num_recommendations = 5

#print("hyb ", user_id, users_df.loc[user_id, 'user_name'], num_recommendations)
user_user_recs = user_user_algo.get_neighbors(user_id, k=num_recommendations)

# Item-Item Collaborative Filtering Recommendations
item_item_recs = item_item_algo.get_neighbors(user_id, k=num_recommendations)

# User-Item Collaborative Filtering Recommendations
user_item_recs = user_item_collaborative_filtering(user_id, num_recommendations)

# Filter the ratings_df DataFrame to get reviewed meals for the user
user_reviews = ratings_df[ratings_df['user_id'] == user_id]

# Get the list of meal IDs reviewed by the user
reviewed_meals = user_reviews['meal_id'].tolist()




#Print original meals reviewed by the user and their ratings
# print("\nOriginal Reviewed Meals and Ratings:")
for meal_id in reviewed_meals:
    #print("\n||||\n", meal_id)
    #print(meals_df[(meals_df['meal_id'] == meal_id)])

    meal_name = meals_df[(meals_df['meal_id'] == meal_id)]["meal_name"].values[0] # meals_df.loc[meal_id, 'meal_name']
    
    user_rating = ratings_df[(ratings_df['user_id'] == user_id) & (ratings_df['meal_id'] == meal_id)]['rating'].values[0]
    
    print(f"Meal: {meal_name} (Meal ID: {meal_id}), Rating: {user_rating}")

# Print User-User Collaborative Filtering Recommendations
print("\nUser-User Collaborative Filtering Recommendations:", user_user_recs)
# for rank, meal_id in enumerate(user_user_recs, start=1):
#     #print(rank, meal_id)
#     print("aaa" + meals_df[meals_df['meal_id'] == meal_id]["meal_name"].values[0])
#     #meal_name = meals_df[meals_df['meal_id'] == meal_id].values[0]  # meals_df.loc[meal_id, 'meal_name'] #meal_name = meals_df[(meals_df['meal_id'] == meal_id)]["meal_name"].values[0] #meal_name = meals_df.loc[meal_id, 'meal_name']
#     #print(f"Rank {rank}: Meal: {meal_name} (Meal ID: {meal_id})")

# Print Item-Item Collaborative Filtering Recommendations
print("\nItem-Item Collaborative Filtering Recommendations:", item_item_recs)
# for rank, meal_id in enumerate(item_item_recs, start=1):
#     #print(rank, meal_id)
#     print("aaa" + meals_df[meals_df['meal_id'] == meal_id]["meal_name"].values[0])
#     #meal_name = meals_df[meals_df['meal_id'] == meal_id].values[0] # meal_name = meals_df.loc[meal_id, 'meal_name'] #meal_name = meals_df[(meals_df['meal_id'] == meal_id)]["meal_name"].values[0] #meal_name = meals_df.loc[meal_id, 'meal_name']
#     #print(f"Rank {rank}: Meal: {meal_name} (Meal ID: {meal_id})")

# Print User-Item Collaborative Filtering Recommendations
print("\nUser-Item Collaborative Filtering Recommendations:", user_item_recs)
# for rank, meal_id in enumerate(user_item_recs, start=1):
#     #print(rank, meal_id)
#     print("aaa" + meals_df[meals_df['meal_id'] == meal_id]["meal_name"].values[0])
#     #meal_name = meals_df[meals_df['meal_id'] == meal_id].values[0] #meal_name = meals_df.loc[meal_id, 'meal_name'] #meal_name = meals_df[(meals_df['meal_id'] == meal_id)]["meal_name"].values[0] #meal_name = meals_df.loc[meal_id, 'meal_name']
#     #print(f"Rank {rank}: Meal: {meal_name} (Meal ID: {meal_id})")


all_meal_ids [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
user_rated_meals [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
remaining_meal_ids []
predictions []
sorted_predictions []
top_recommendations []
Meal: Spaghetti Bolognese (Meal ID: 1), Rating: 1
Meal: Chicken Caesar Salad (Meal ID: 2), Rating: 3
Meal: Pepperoni Pizza (Meal ID: 3), Rating: 1
Meal: Sushi Platter (Meal ID: 4), Rating: 1
Meal: Vegetable Stir-Fry (Meal ID: 5), Rating: 4
Meal: Grilled Salmon (Meal ID: 6), Rating: 2
Meal: Margarita Pizza (Meal ID: 7), Rating: 1
Meal: Beef Tacos (Meal ID: 8), Rating: 4
Meal: Chicken Alfredo (Meal ID: 9), Rating: 4
Meal: Caesar Salad (Meal ID: 10), Rating: 3
Meal: Chicken Teriyaki Stir-Fry (Meal ID: 

Auto generating sample ratings data

In [9]:

# import random

# # Num users, meals
# num_users = 100
# num_meals = 50

# # Generate random ratings
# ratings = []
# for user_id in range(1, num_users + 1):
#     for meal_id in range(1, num_meals + 1):
#         rating = random.randint(1, 5)  # Random rating between 1 and 5
#         ratings.append([user_id, meal_id, rating])

# # Save the ratings to a CSV file
# ratings_df = pd.DataFrame(ratings, columns=['user_id', 'meal_id', 'rating'])
# ratings_df.to_csv('ratings.csv', index=False)
